# Zarządzanie ryzykiem w firmie NVIDIA

Szymon Malec <br>
Michał Wiktorowski <br>
Filip Oszczepaliński <br>
Damian Szuster

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as sp

__Zmienne ryzyka__:
- cena miedzi
- cena aluminium